DATA IMPORT

PARAGRAPH SPLITTING

In [2]:
import os
import pandas as pd
import json
from pandas import json_normalize
import re
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import classification_report, f1_score
from sklearn.metrics.pairwise import cosine_distances
import sys

import torch

import os
#sys.path.append(os.path.abspath("pan-code/clef22/style-change-detection/evaluator"))
#!pip install evaluator
#from evaluator import *
from sentence_transformers import SentenceTransformer, InputExample, util, losses
from transformers import AutoTokenizer
from torch.utils.data import Dataset, DataLoader
from typing import List
import random
import math

In [4]:
import os
import json
import pandas as pd

# Specify the folder containing the JSON and text files
folder_path = r"C:\Users\EBRAHIMR2\Downloads\Data Science Masters\Modules\COS 801\pan23-multi-author-analysis-dataset1\pan23-multi-author-analysis-dataset1-train"

# Lists to store data
json_data_list = []
text_data_list = []
authors_list = []
changes_list = []

# Loop through files in the folder
for file_name in os.listdir(folder_path):
    if file_name.startswith("truth-problem-") and file_name.endswith(".json"):
        json_file_path = os.path.join(folder_path, file_name)
        
        # Extract the post number from the JSON file name
        post_number = int(file_name.split("-")[-1].split(".")[0])
        
        # Construct the corresponding text file name
        text_file_name = f"problem-{post_number}.txt"
        text_file_path = os.path.join(folder_path, text_file_name)
        
        if os.path.exists(json_file_path) and os.path.exists(text_file_path):

            # Read JSON data
            with open(json_file_path, "r", encoding="utf-8") as json_file:
                json_data = json.load(json_file)
                json_data_list.append(json_data)
                
                # Extract authors and changes from JSON
                authors_list.append(json_data['authors'])
                changes_list.append(json_data['changes'])
            
            # Read text file
            with open(text_file_path, "r", encoding="utf-8") as text_file:
                text_data = text_file.read()
                text_data_list.append(text_data)

# Create a DataFrame
data = pd.DataFrame({'Post Number': range(1, len(json_data_list) + 1),
                     'JSON Data': json_data_list,
                     'Authors': authors_list,
                     'Changes': changes_list,
                     'Text Data': text_data_list})

# Function to split text into paragraphs
def split_into_paragraphs(text):
    return text.splitlines()

# Apply the function to split text into paragraphs and add as a new column
data['splitted_text'] = data['Text Data'].apply(split_into_paragraphs)

# Print the first few rows of the DataFrame
print(data.head())

# Export the DataFrame to an Excel file
data.to_excel("output_data1.xlsx", index=False)



   Post Number                                JSON Data  Authors  \
0            1        {'authors': 2, 'changes': [1, 0]}        2   
1            2  {'authors': 3, 'changes': [1, 1, 0, 1]}        3   
2            3     {'authors': 3, 'changes': [1, 1, 1]}        3   
3            4           {'authors': 2, 'changes': [1]}        2   
4            5     {'authors': 4, 'changes': [1, 1, 1]}        4   

        Changes                                          Text Data  \
0        [1, 0]  I'm not arguing with you here, I'm simply tryi...   
1  [1, 1, 0, 1]  Biden has the power to write an executive orde...   
2     [1, 1, 1]  In general, be courteous to others. Debate/dis...   
3           [1]  Second, until late in the 20th century, the Ot...   
4     [1, 1, 1]  Arming and supplying a generation of hostile y...   

                                       splitted_text  
0  [I'm not arguing with you here, I'm simply try...  
1  [Biden has the power to write an executive ord...  
2  [I

In [5]:

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [6]:
def set_reproducibility(seed):
  random.seed(seed)
  np.random.seed(seed)
  
  # Pytorch
  torch.use_deterministic_algorithms(True)
  torch.manual_seed(seed)

In [7]:
from typing import List
from transformers import AutoTokenizer
from sentence_transformers import InputExample
import pandas as pd
import numpy as np

def create_contrastive_examples(df: pd.DataFrame, tokenizer: AutoTokenizer, max_length: int, max_samples: int):
    rows = []
    
    for i in range(len(df)):
        # Assuming 'splitted_text' is the column containing your text data
        document = df['splitted_text'].iloc[i]
        
        # Assuming 'Authors' is the column containing author information
        authors = df['Authors'].iloc[i]

        if len(authors) != len(document):
            continue

        for i in range(len(document)):
            for j in range(len(document)):
                if i != j:
                    new_row = {}

                    text1 = document[i][:sum([len(t) + 1 for t in tokenizer.tokenize(document[i])[:max_length]])]
                    text2 = document[j][:sum([len(t) + 1 for t in tokenizer.tokenize(document[j])[:max_length]])]

                    new_row["text1"] = text1
                    new_row["text2"] = text2

                    if authors[i] != authors[j]:
                        new_row["label"] = 0  # 0 if different
                    else:
                        new_row["label"] = 1  # 1 if same

                    rows.append(new_row)

    df = pd.DataFrame(rows, columns=['text1', 'text2', 'label'])

    # Sample the required positive and negative samples
    pos_samples = df[df["label"] == 1]
    neg_samples = df[df["label"] == 0]
    
    if len(pos_samples) > max_samples // 2:
        pos_samples = pos_samples.sample(max_samples // 2)
    if len(neg_samples) > max_samples // 2:
        neg_samples = neg_samples.sample(max_samples // 2)    

    df = pd.concat([pos_samples, neg_samples], axis=0, ignore_index=True).sample(frac=1)

    # From this create a dataset which can be handled by sentence-transformers
    train_examples = []
    for i, row in df.iterrows():
        train_examples.append(InputExample(texts=[row["text1"], row["text2"]], label=row["label"]))

    return np.array(train_examples)

# Example usage:
# train_examples = create_contrastive_examples(your_dataframe, your_tokenizer, your_max_length, your_max_samples)


In [8]:
from transformers import AutoTokenizer
from sentence_transformers import InputExample
import pandas as pd
import numpy as np

def create_triplet_examples(df, tokenizer, max_length, n_per_document, max_samples, model, margin, mine_all=True, mine_hard=False, mine_semihard=False, is_task1=False):
    rows = []
    rows_hard = []
    rows_shard = []
    
    # Assuming 'splitted_text' is the column containing your text data
    documents = df['splitted_text']
    
    # Assuming 'Authors' is the column containing author information
    authors = df['Authors']
    
    # Your dataset structure might differ, adjust accordingly

    for i in range(len(df)):
        used_idxs = []
        n_remaining = n_per_document

        # Get the indexes of each author
        author_idxs = {}
        for index, value in enumerate(authors[i]):
            if value not in author_idxs:
                author_idxs[value] = []
            author_idxs[value].append(index)

        if is_task1:
            # Take the hard example (the change one)
            changes = [int(authors[i][j] == authors[i][j+1]) for j in range(len(authors[i])-1)]
            change_idx = changes.index(0)

            try:
                a_idx = random.choice([x for x in author_idxs[authors[i][change_idx]] if x != change_idx])
            except IndexError:
                continue
            pos_idx = change_idx
            neg_idx = change_idx + 1

            new_row = {}

            new_row["anchor"] = documents[i][a_idx][:sum([len(t) + 1 for t in tokenizer.tokenize(documents[i][a_idx])[:max_length]])]
            new_row["pos"] = documents[i][pos_idx][:sum([len(t) + 1 for t in tokenizer.tokenize(documents[i][pos_idx])[:max_length]])]
            new_row["neg"] = documents[i][neg_idx][:sum([len(t) + 1 for t in tokenizer.tokenize(documents[i][neg_idx])[:max_length]])]

            rows_shard.append(new_row)
            used_idxs.append((a_idx, pos_idx, neg_idx))

            n_remaining -= 1
        else:
            if all([len(author_idxs[k]) == 1 for k in author_idxs]):
                continue

        # I could compute how many combinations are possible but it's too hard
        idx_errors = 0
        while n_remaining > 0:
            if idx_errors > 5:
                break
            try:
                # Sample a random author
                anchor_author = random.choice(list(author_idxs.keys()))

                # Get a paragraph from that author (anchor)
                a_idx = random.choice(author_idxs[anchor_author])

                # Get another paragraph from the same author
                pos_idx = random.choice([x for x in author_idxs[anchor_author] if x != a_idx])

                # Sample another author
                neg_author = random.choice([x for x in list(author_idxs.keys()) if x != anchor_author])

                # Get a paragraph from that author
                neg_idx = random.choice(author_idxs[neg_author])
            except IndexError:
                idx_errors += 1
                continue

            if (a_idx, pos_idx, neg_idx) in used_idxs:
                idx_errors += 1
                continue
            else:
                used_idxs.append((a_idx, pos_idx, neg_idx))
                new_row = {}

                new_row["anchor"] = documents[i][a_idx][:sum([len(t) + 1 for t in tokenizer.tokenize(documents[i][a_idx])[:max_length]])]
                new_row["pos"] = documents[i][pos_idx][:sum([len(t) + 1 for t in tokenizer.tokenize(documents[i][pos_idx])[:max_length]])]
                new_row["neg"] = documents[i][neg_idx][:sum([len(t) + 1 for t in tokenizer.tokenize(documents[i][neg_idx])[:max_length]])]

                if mine_hard or mine_semihard:
                    # Compute embeddings
                    a_embedding = model.encode([new_row["anchor"]], convert_to_tensor=True)
                    p_embedding = model.encode([new_row["pos"]], convert_to_tensor=True)
                    n_embedding = model.encode([new_row["neg"]], convert_to_tensor=True)

                    # Compute distance between anchor and positive
                    a_p_dist = 1 - util.cos_sim(a_embedding, p_embedding).cpu().item()
                    # Compute distance between anchor and negative
                    a_n_dist = 1 - util.cos_sim(a_embedding, n_embedding).cpu().item()

                    if a_n_dist < a_p_dist:
                        # Hard example
                        if mine_hard:
                            rows_hard.append(new_row)
                    elif a_n_dist < a_p_dist + margin and a_n_dist > a_p_dist:
                        # Semi hard example
                        if mine_semihard:
                            rows_shard.append(new_row)
                    else:
                        if mine_all:
                            rows.append(new_row)
                    n_remaining -= 1
                else:
                    rows.append(new_row)
                    n_remaining -= 1

    print(f"N easy:{len(rows)}, N hard: {len(rows_hard)}, N semihard:{len(rows_shard)}")
    random.shuffle(rows_shard)
    random.shuffle(rows_hard)
    random.shuffle(rows)

    all_rows = [*rows_shard, *rows_hard, *rows]
    if len(all_rows) < max_samples:
        all_rows = all_rows[:max_samples]
    print(len(all_rows))

    df = pd.DataFrame(all_rows, columns=['anchor', 'pos', 'neg'])

    # From this create a dataset which can be handled by sentence-transformers
# From this create a dataset which can be handled by sentence-transformers
# From this create a dataset which can be handled by sentence-transformers
    train_examples = []
    for i, row in df.iterrows():
        train_examples.append(InputExample(texts=[row["anchor"], row["pos"], row["neg"]]))

    return np.array(train_examples)


In [9]:
def compute_same_diff_contrastive(model, dataloader):
    model.eval()
    same_sims = []
    diff_sims = []

    for idx, batch in enumerate(dataloader):
        with torch.no_grad():
            text1 = {k: batch[0][0][k].to(device) for k in batch[0][0]}
            embeddings1 = model(text1)["sentence_embedding"]

            text2 = {k: batch[0][1][k].to(device) for k in batch[0][1]}
            embeddings2 = model(text2)["sentence_embedding"]

            author_info = batch[1]

            for i in range(len(author_info)):
                same = author_info[i]
                sim = util.cos_sim(embeddings1[i], embeddings2[i]).cpu().item()
                if same == 1:
                    same_sims.append(sim)
                else:
                    diff_sims.append(sim)
    return same_sims, diff_sims

def compute_same_diff_triplet(model, dataloader):
    model.eval()
    same_sims = []
    diff_sims = []

    for idx, batch in enumerate(dataloader):
        with torch.no_dataloader():
            anchor = {k: batch[0][0][k].to(device) for k in batch[0][0]}
            embeddings_a = model(anchor)["sentence_embedding"]

            pos = {k: batch[0][1][k].to(device) for k in batch[0][1]}
            embeddings_p = model(pos)["sentence_embedding"]

            neg = {k: batch[0][2][k].to(device) for k in batch[0][2]}
            embeddings_n = model(neg)["sentence_embedding"]

            for i in range(anchor["input_ids"].shape[0]):
                sim_pos = util.cos_sim(embeddings_a[i], embeddings_p[i]).cpu().item()
                sim_neg = util.cos_sim(embeddings_a[i], embeddings_n[i]).cpu().item()

                same_sims.append(sim_pos)
                diff_sims.append(sim_neg)

    return same_sims, diff_sims


In [13]:
from sentence_transformers import SentenceTransformer, util

def predict_author_change(model, paragraphs):
    # Ensure your model is in evaluation mode
    model.eval()

    # Initialize a list to store the similarity scores
    similarity_scores = []

    # Loop through successive pairs of passages
    for i in range(1, len(paragraphs)):
        # Encode the passages using the model
        passage1_embedding = model.encode(paragraphs[i - 1], convert_to_tensor=True)
        passage2_embedding = model.encode(paragraphs[i], convert_to_tensor=True)

        # Compute the cosine similarity between the embeddings
        similarity = util.pytorch_cos_sim(passage1_embedding, passage2_embedding)
        
        # Append the similarity score to the list
        similarity_scores.append(similarity.item())

    # Define a threshold for change detection
    threshold = 0.5  # You can adjust this threshold as needed

    # Check if any of the similarity scores indicate a change of author
    author_change = any(score < threshold for score in similarity_scores)

    return author_change

# Example usage:
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
paragraphs = [
    
    "He's at my place half the time and his fiance(e)'s place the other half of the time. He's been (homeless) couch surfing for several years and only recently got engaged to his other partner. We don't have any current issues that would lead me to want this arrangement to stop, but I do want to protect my own legal rights.",
    "I don't think he, in particular, would do that, but I do want to retain my own legal rights wherever possible/appropriate.",
  
]

author_change = predict_author_change(model, paragraphs)
print("Author change detected:", author_change)


Author change detected: False


In [15]:
#DONT RUN

from sentence_transformers import SentenceTransformer, util
import pandas as pd

# Load your DataFrame with the paragraphs (Replace 'df' with your DataFrame)
df = pd.read_excel("output_data.xlsx")  # Load your data

# Sample 10% of your dataset
sampled_df = df.sample(frac=0.1, random_state=42)  # Adjust the random state as needed

# Initialize the Sentence Transformer model (you can use your own model)
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Function to predict author changes between paragraphs
def predict_author_change(model, paragraphs):
    # Ensure your model is in evaluation mode
    model.eval()

    # Initialize a list to store the similarity scores
    similarity_scores = []

    # Loop through successive pairs of paragraphs
    for i in range(1, len(paragraphs)):
        # Encode the paragraphs using the model
        passage1_embedding = model.encode(paragraphs[i - 1], convert_to_tensor=True)
        passage2_embedding = model.encode(paragraphs[i], convert_to_tensor=True)

        # Compute the cosine similarity between the embeddings
        similarity = util.pytorch_cos_sim(passage1_embedding, passage2_embedding)
        
        # Append the similarity score to the list
        similarity_scores.append(similarity.item())

    # Define a threshold for change detection
    threshold = 0.5  # You can adjust this threshold as needed

    # Check if any of the similarity scores indicate a change of author
    author_change = any(score < threshold for score in similarity_scores)

    return author_change

# Apply the prediction function to your sampled DataFrame
predictions = []
for index, row in sampled_df.iterrows():
    paragraphs = row['splitted_text']  # Modify this based on your DataFrame column
    author_change = predict_author_change(model, paragraphs)
    predictions.append(author_change)

# Add the predictions as a new column in your sampled DataFrame
sampled_df['Predicted_Author_Change'] = predictions

# You can save the updated DataFrame or further analyze the results as needed


In [16]:
#sampled_df


,Post Number,JSON Data,Authors,Changes,Text Data,splitted_text,Predicted_Author_Change
1743,1744,"{'authors': 4, 'changes': [1, 1, 1, 1, 1]}",4,"[1, 1, 1, 1, 1]","For so many years, decades even, russia has be...","['For so many years, decades even, russia has ...",True
2196,2197,"{'authors': 2, 'changes': [1]}",2,[1],Now an archer can only aim at one target at a ...,['Now an archer can only aim at one target at ...,True
1728,1729,"{'authors': 2, 'changes': [0, 1]}",2,"[0, 1]",What the attorney means by this is that they'r...,"[""What the attorney means by this is that they...",True
3337,3338,"{'authors': 4, 'changes': [1, 1, 1, 1, 1, 1, 1...",4,"[1, 1, 1, 1, 1, 1, 1, 0, 1]","In general, be courteous to others. Debate/dis...","[""In general, be courteous to others. Debate/d...",True
298,299,"{'authors': 2, 'changes': [1, 1]}",2,"[1, 1]",Schools all over the world will deny enrollmen...,['Schools all over the world will deny enrollm...,True
...,...,...,...,...,...,...,...
3949,3950,"{'authors': 2, 'changes': [0, 1, 0]}",2,"[0, 1, 0]",Can you explain? By headers I assume they mean...,['Can you explain? By headers I assume they me...,True
174,175,"{'authors': 4, 'changes': [1, 1, 1]}",4,"[1, 1, 1]",Yep. Had a gun runner case a while back. He wa...,['Yep. Had a gun runner case a while back. He ...,True
393,394,"{'authors': 2, 'changes': [0, 1]}",2,"[0, 1]",You can use the will to nominate a guardian. I...,['You can use the will to nominate a guardian....,True
3721,3722,"{'authors': 2, 'changes': [1, 0, 0, 1]}",2,"[1, 0, 0, 1]","I am a bot, and this action was performed auto...","['I am a bot, and this action was performed au...",True


In [ ]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd

# Load your DataFrame with the paragraphs (Replace 'df' with your DataFrame)
df = pd.read_excel("output_data.xlsx")  # Load your data

# Sample 10% of your dataset
sampled_df = df.sample(frac=0.1, random_state=42)  # Adjust the random state as needed

# Initialize the Sentence Transformer model (you can use your own model)
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Function to predict author changes between paragraphs
def predict_author_change(model, paragraphs):
    # Ensure your model is in evaluation mode
    model.eval()

    # Initialize a list to store the similarity scores
    similarity_scores = []

    # Loop through successive pairs of paragraphs
    for i in range(1, len(paragraphs)):
        # Encode the paragraphs using the model
        passage1_embedding = model.encode(paragraphs[i - 1], convert_to_tensor=True)
        passage2_embedding = model.encode(paragraphs[i], convert_to_tensor=True)

        # Compute the cosine similarity between the embeddings
        similarity = util.pytorch_cos_sim(passage1_embedding, passage2_embedding)
        
        # Append the similarity score to the list
        similarity_scores.append(similarity.item())

    # Define a threshold for change detection
    threshold = 0.5  # You can adjust this threshold as needed

    # Check if any of the similarity scores indicate a change between paragraphs
    author_changes = [score < threshold for score in similarity_scores]

    return author_changes

# Apply the prediction function to your sampled DataFrame
predictions = []
for index, row in sampled_df.iterrows():
    paragraphs = row['splitted_text']  # Modify this based on your DataFrame column
    author_changes = predict_author_change(model, paragraphs)
    predictions.append(author_changes)

# Add the predictions as a new column in your sampled DataFrame
sampled_df['Predicted_Author_Changes'] = predictions

# You can save the updated DataFrame or further analyze the results as needed


              precision    recall  f1-score   support

           0       0.40      0.40      0.40         5
           1       0.40      0.40      0.40         5

    accuracy                           0.40        10
   macro avg       0.40      0.40      0.40        10
weighted avg       0.40      0.40      0.40        10



In [32]:
from sklearn.metrics import classification_report

def prediction_task3_threshold(model : SentenceTransformer, passages : List[str], threshold : float):
    """
    Perform inference on a set of passages using the threshold approach
    """
    embeddings = model.encode(passages, convert_to_tensor=True)

    similarity_scores = []

    for i in range(len(passages) - 1):
        sim = util.cos_sim(embeddings[i], embeddings[i + 1])
        similarity_scores.append(sim.item())

    # Threshold
    pred_changes = [1 if sim < threshold else 0 for sim in similarity_scores]
    return pred_changes

def evaluate_model_task3_threshold(model : SentenceTransformer, df : pd.DataFrame, threshold : float, save_pred_to_disk : str = None):
    """
    Evaluate a SentenceTransformer encoder model using the task3 inference method with the threshold approach on a dataframe of examples
    """
    valid_labels = []
    y_preds = []

    for i, row in df.iterrows():
        passages = row["splitted_text"]

        pred_authors = prediction_task3_threshold(model, passages, threshold)

        valid_labels += row["changes"]
        y_preds += pred_authors

    print(classification_report(valid_labels, y_preds))
    if not save_pred_to_disk is None:
        np.save(save_pred_to_disk, y_preds)



Data has been exported to data_with_paragraphs.xlsx


      Post Number                                JSON Data  Authors  \
0               1        {'authors': 2, 'changes': [1, 0]}        2   
1              10  {'authors': 3, 'changes': [1, 1, 0, 1]}        3   
2             100     {'authors': 3, 'changes': [1, 1, 1]}        3   
3            1000           {'authors': 2, 'changes': [1]}        2   
4            1001     {'authors': 4, 'changes': [1, 1, 1]}        4   
...           ...                                      ...      ...   
4195          995        {'authors': 2, 'changes': [1, 1]}        2   
4196          996           {'authors': 2, 'changes': [1]}        2   
4197          997     {'authors': 4, 'changes': [1, 1, 1]}        4   
4198          998        {'authors': 3, 'changes': [1, 1]}        3   
4199          999        {'authors': 3, 'changes': [1, 1]}        3   

           Changes                                          Text Data  \
0           [1, 0]  I'm not arguing with you here, I'm simply tryi...   
1

,Post Number,JSON Data,Authors,Changes,Text Data,Paragraphs List
0,1,"{'authors': 2, 'changes': [1, 0]}",2,"[1, 0]","I'm not arguing with you here, I'm simply tryi...","[I'm not arguing with you here, I'm simply try..."
1,10,"{'authors': 3, 'changes': [1, 1, 0, 1]}",3,"[1, 1, 0, 1]",Biden has the power to write an executive orde...,[Biden has the power to write an executive ord...
2,100,"{'authors': 3, 'changes': [1, 1, 1]}",3,"[1, 1, 1]","In general, be courteous to others. Debate/dis...","[In general, be courteous to others. Debate/di..."
3,1000,"{'authors': 2, 'changes': [1]}",2,[1],"Second, until late in the 20th century, the Ot...","[Second, until late in the 20th century, the O..."
4,1001,"{'authors': 4, 'changes': [1, 1, 1]}",4,"[1, 1, 1]",Arming and supplying a generation of hostile y...,[Arming and supplying a generation of hostile ...


Accuracy: 0.7452380952380953
              precision    recall  f1-score   support

           0       0.78      0.03      0.06       219
           1       1.00      1.00      1.00       840

   micro avg       1.00      0.80      0.89      1059
   macro avg       0.89      0.52      0.53      1059
weighted avg       0.95      0.80      0.81      1059
 samples avg       1.00      0.87      0.92      1059



   Post Number  Paragraph Number  \
0            1                 1   
1            1                 2   
2            2                 1   
3            2                 2   
4            2                 3   

                                      Paragraph Text  Authors  Label  
0  I'm not arguing with you here, I'm simply tryi...        2      1  
1  He's at my place half the time and his fiance(...        2      0  
2  Biden has the power to write an executive orde...        3      1  
3  r/politics is currently accepting new moderato...        3      1  
4  The inflation reduction act increases inflatio...        3      0  


,Post Number,Paragraph Number,Paragraph Text,Authors,Label
0,1,1,"I'm not arguing with you here, I'm simply tryi...",2,1
1,1,2,He's at my place half the time and his fiance(...,2,0
2,2,1,Biden has the power to write an executive orde...,3,1
3,2,2,r/politics is currently accepting new moderato...,3,1
4,2,3,The inflation reduction act increases inflatio...,3,0
5,2,4,There is really only one solution that is work...,3,1
6,3,1,"In general, be courteous to others. Debate/dis...",3,1
7,3,2,"Are you any of these? If so, then don't be so ...",3,1
8,3,3,r/politics is currently accepting new moderato...,3,1
9,4,1,"Second, until late in the 20th century, the Ot...",2,1


Post Number	Paragraph Number	Paragraph Text	Authors	Label
   Post Number  Paragraph Number  \
0            1                 1   
1            1                 2   
2            2                 1   
3            2                 2   
4            2                 3   

                                      Paragraph Text  Authors  Label  
0  I'm not arguing with you here, I'm simply tryi...        2      0  
1  He's at my place half the time and his fiance(...        2      1  
2  Biden has the power to write an executive orde...        3      0  
3  r/politics is currently accepting new moderato...        3      1  
4  The inflation reduction act increases inflatio...        3      1  


,Post Number,Paragraph Number,Paragraph Text,Authors,Label
0,1,1,"I'm not arguing with you here, I'm simply tryi...",2,0
1,1,2,He's at my place half the time and his fiance(...,2,1
2,2,1,Biden has the power to write an executive orde...,3,0
3,2,2,r/politics is currently accepting new moderato...,3,1
4,2,3,The inflation reduction act increases inflatio...,3,1
5,2,4,There is really only one solution that is work...,3,0
6,3,1,"In general, be courteous to others. Debate/dis...",3,0
7,3,2,"Are you any of these? If so, then don't be so ...",3,1
8,3,3,r/politics is currently accepting new moderato...,3,1
9,4,1,"Second, until late in the 20th century, the Ot...",2,0


PROCESSING WITH LABELS CORRECT

Post Number	Paragraph Number	Paragraph Text	Authors	Label
   Post Number  Paragraph Number  \
0            1                 1   
1            1                 2   
2            1                 3   
3            2                 1   
4            2                 2   

                                      Paragraph Text  Authors  Label  
0  I'm not arguing with you here, I'm simply tryi...        2      0  
1  He's at my place half the time and his fiance(...        2      1  
2  I don't think he, in particular, would do that...        2      0  
3  Biden has the power to write an executive orde...        3      0  
4  r/politics is currently accepting new moderato...        3      1  


,Post Number,Paragraph Number,Paragraph Text,Authors,Label
0,1,1,"I'm not arguing with you here, I'm simply tryi...",2,0
1,1,2,He's at my place half the time and his fiance(...,2,1
2,1,3,"I don't think he, in particular, would do that...",2,0
3,2,1,Biden has the power to write an executive orde...,3,0
4,2,2,r/politics is currently accepting new moderato...,3,1
5,2,3,The inflation reduction act increases inflatio...,3,1
6,2,4,There is really only one solution that is work...,3,0
7,2,5,You aren’t given unrealistic expectations to p...,3,1
8,3,1,"In general, be courteous to others. Debate/dis...",3,0
9,3,2,"Are you any of these? If so, then don't be so ...",3,1


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17104 entries, 0 to 17103
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Post Number       17104 non-null  int64 
 1   Paragraph Number  17104 non-null  int64 
 2   Paragraph Text    17104 non-null  object
 3   Authors           17104 non-null  int64 
 4   Label             17104 non-null  int64 
dtypes: int64(4), object(1)
memory usage: 668.2+ KB


     ---------------------------------------- 0.0/42.3 kB ? eta -:--:--
     ---------------------------------------- 42.3/42.3 kB 2.0 MB/s eta 0:00:00
     ---------------------------------------- 0.0/43.6 kB ? eta -:--:--
     ---------------------------------------- 43.6/43.6 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 0.0/977.5 kB ? eta -:--:--
     --- ---------------------------------- 102.4/977.5 kB 2.9 MB/s eta 0:00:01
     -------------- ----------------------- 368.6/977.5 kB 4.6 MB/s eta 0:00:01
     ------------------------ ------------- 624.6/977.5 kB 4.9 MB/s eta 0:00:01
     --------------------------- ---------- 696.3/977.5 kB 4.9 MB/s eta 0:00:01
     -------------------------------------  972.8/977.5 kB 4.7 MB/s eta 0:00:01
     -------------------------------------- 977.5/977.5 kB 4.4 MB/s eta 0:00:00


  ERROR: HTTP error 403 while getting https://files.pythonhosted.org/packages/e7/7f/470d6fcdf23f9f3518f6b0b76be9df16dcc8630ad409947f8be2eb0ed13a/pathtools-0.1.2.tar.gz (from https://pypi.org/simple/pathtools/)
ERROR: Could not install requirement pathtools from https://files.pythonhosted.org/packages/e7/7f/470d6fcdf23f9f3518f6b0b76be9df16dcc8630ad409947f8be2eb0ed13a/pathtools-0.1.2.tar.gz (from wandb>=0.10.32->simpletransformers) because of HTTP error 403 Client Error: Forbidden for url: https://files.pythonhosted.org/packages/e7/7f/470d6fcdf23f9f3518f6b0b76be9df16dcc8630ad409947f8be2eb0ed13a/pathtools-0.1.2.tar.gz for URL https://files.pythonhosted.org/packages/e7/7f/470d6fcdf23f9f3518f6b0b76be9df16dcc8630ad409947f8be2eb0ed13a/pathtools-0.1.2.tar.gz (from https://pypi.org/simple/pathtools/)


  Obtaining dependency information for pip from https://files.pythonhosted.org/packages/47/6a/453160888fab7c6a432a6e25f8afe6256d0d9f2cbd25971021da6491d899/pip-23.3.1-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   -- ------------------------------------- 0.2/2.1 MB 3.1 MB/s eta 0:00:01
   --------- ------------------------------ 0.5/2.1 MB 5.0 MB/s eta 0:00:01
   ------------ --------------------------- 0.7/2.1 MB 4.6 MB/s eta 0:00:01
   ------------ --------------------------- 0.7/2.1 MB 4.6 MB/s eta 0:00:01
   ---------------------- ----------------- 1.2/2.1 MB 4.9 MB/s eta 0:00:01
   ------------------------ --------------- 1.3/2.1 MB 4.9 MB/s eta 0:00:01
   ------------------------ --------------- 1.3/2.1 MB 4.9 MB/s eta 0:00:01
   ------------------------------- -------- 1.7/2.1 MB 4.9 MB/s eta 0:00:01
   ------------------------------------ --- 1.9/2.1 MB 4.5 MB/s eta 0:00:01
   ---------------------------------------- 2.1/2.1 MB 

DefaultVerifyPaths(cafile=None, capath=None, openssl_cafile_env='SSL_CERT_FILE', openssl_cafile='C:\\Program Files\\Common Files\\ssl/cert.pem', openssl_capath_env='SSL_CERT_DIR', openssl_capath='C:\\Program Files\\Common Files\\ssl/certs')


C:\Users\EBRAHIMR2\AppData\Local\anaconda3\lib\site-packages\huggingface_hub\file_download.py:137: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\EBRAHIMR2\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-small-discrim

Map:   0%|          | 0/11972 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/5132 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


ValueError: The model did not return a loss from the inputs, only the following keys: logits. For reference, the inputs it received are input_ids,token_type_ids,attention_mask.

DataFrame exported to C:\Users\EBRAHIMR2\Downloads\Data Science Masters\Modules\COS 801\pan23-multi-author-analysis-dataset1\pan23-multi-author-analysis-dataset1-train\output.xlsx


,Post Number,Paragraph Number,Paragraph Text,Label
0,1,1,"I'm not arguing with you here, I'm simply tryi...",1
1,1,2,He's at my place half the time and his fiance(...,1
2,1,3,"I don't think he, in particular, would do that...",1
3,2,1,Biden has the power to write an executive orde...,1
4,2,2,r/politics is currently accepting new moderato...,1


,Post Number,Paragraph Number,Paragraph Text,Label
0,1,1,"I'm not arguing with you here, I'm simply tryi...",1.0
1,1,2,He's at my place half the time and his fiance(...,NaN
2,1,3,"I don't think he, in particular, would do that...",NaN
3,2,1,Biden has the power to write an executive orde...,1.0
4,2,2,r/politics is currently accepting new moderato...,NaN


,Post Number,Paragraph Number,Paragraph Text,Label,Changes
0,1,1,"I'm not arguing with you here, I'm simply tryi...",1,None
1,1,2,He's at my place half the time and his fiance(...,0,None
2,1,3,"I don't think he, in particular, would do that...",0,None
3,2,1,Biden has the power to write an executive orde...,1,None
4,2,2,r/politics is currently accepting new moderato...,0,None


Document 1 Paragraphs:
Paragraph 1:
I'm not arguing with you here, I'm simply trying to contextualize this for you. To the extent that they are there, it is with your consent. The state has passed laws making sure that vulnerable people (not saying he's one) don't get abused (not saying you're abusing him), and in casting a wide net to save as many vulnerable little birds as possible from hitting the floor after being kicked out of their nest wrongfully, the state has (as much from a lack of better options as from any other reason) created a circumstance where occasionally some not-so-vulnerable little bird can take advantage of someone else's nest.
He's at my place half the time and his fiance(e)'s place the other half of the time. He's been (homeless) couch surfing for several years and only recently got engaged to his other partner. We don't have any current issues that would lead me to want this arrangement to stop, but I do want to protect my own legal rights.
I don't think he, in

Total Number of Posts: 4200


   Post Number                                JSON Data  \
0            1        {'authors': 2, 'changes': [1, 0]}   
1            2  {'authors': 3, 'changes': [1, 1, 0, 1]}   
2            3     {'authors': 3, 'changes': [1, 1, 1]}   
3            4           {'authors': 2, 'changes': [1]}   
4            5     {'authors': 4, 'changes': [1, 1, 1]}   

                                           Text Data  
0  I'm not arguing with you here, I'm simply tryi...  
1  Biden has the power to write an executive orde...  
2  In general, be courteous to others. Debate/dis...  
3  Second, until late in the 20th century, the Ot...  
4  Arming and supplying a generation of hostile y...  


,Post Number,JSON Data,Text Data
0,1,"{'authors': 2, 'changes': [1, 0]}","I'm not arguing with you here, I'm simply tryi..."
1,2,"{'authors': 3, 'changes': [1, 1, 0, 1]}",Biden has the power to write an executive orde...
2,3,"{'authors': 3, 'changes': [1, 1, 1]}","In general, be courteous to others. Debate/dis..."
3,4,"{'authors': 2, 'changes': [1]}","Second, until late in the 20th century, the Ot..."
4,5,"{'authors': 4, 'changes': [1, 1, 1]}",Arming and supplying a generation of hostile y...
...,...,...,...
4195,4196,"{'authors': 2, 'changes': [1, 1]}","Yeah fair enough, I kind of figured I was prob..."
4196,4197,"{'authors': 2, 'changes': [1]}",No need for more suffering. Being a russian is...
4197,4198,"{'authors': 4, 'changes': [1, 1, 1]}",- - - - - - - - - - - - - - - - - - - - - - - ...
4198,4199,"{'authors': 3, 'changes': [1, 1]}",Kanye is being manipulated and taken advantage...


Total Number of Posts: 4200


   Post Number                                JSON Data  Authors  \
0            1        {'authors': 2, 'changes': [1, 0]}        2   
1            2  {'authors': 3, 'changes': [1, 1, 0, 1]}        3   
2            3     {'authors': 3, 'changes': [1, 1, 1]}        3   
3            4           {'authors': 2, 'changes': [1]}        2   
4            5     {'authors': 4, 'changes': [1, 1, 1]}        4   

        Changes                                          Text Data  
0        [1, 0]  I'm not arguing with you here, I'm simply tryi...  
1  [1, 1, 0, 1]  Biden has the power to write an executive orde...  
2     [1, 1, 1]  In general, be courteous to others. Debate/dis...  
3           [1]  Second, until late in the 20th century, the Ot...  
4     [1, 1, 1]  Arming and supplying a generation of hostile y...  


,Post Number,JSON Data,Authors,Changes,Text Data
0,1,"{'authors': 2, 'changes': [1, 0]}",2,"[1, 0]","I'm not arguing with you here, I'm simply tryi..."
1,2,"{'authors': 3, 'changes': [1, 1, 0, 1]}",3,"[1, 1, 0, 1]",Biden has the power to write an executive orde...
2,3,"{'authors': 3, 'changes': [1, 1, 1]}",3,"[1, 1, 1]","In general, be courteous to others. Debate/dis..."
3,4,"{'authors': 2, 'changes': [1]}",2,[1],"Second, until late in the 20th century, the Ot..."
4,5,"{'authors': 4, 'changes': [1, 1, 1]}",4,"[1, 1, 1]",Arming and supplying a generation of hostile y...
...,...,...,...,...,...
4195,4196,"{'authors': 2, 'changes': [1, 1]}",2,"[1, 1]","Yeah fair enough, I kind of figured I was prob..."
4196,4197,"{'authors': 2, 'changes': [1]}",2,[1],No need for more suffering. Being a russian is...
4197,4198,"{'authors': 4, 'changes': [1, 1, 1]}",4,"[1, 1, 1]",- - - - - - - - - - - - - - - - - - - - - - - ...
4198,4199,"{'authors': 3, 'changes': [1, 1]}",3,"[1, 1]",Kanye is being manipulated and taken advantage...


array([2, 3, 4], dtype=int64)

   Author Count  Count
0             2   1400
1             3   1400
2             4   1400


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4200 entries, 0 to 4199
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Post Number  4200 non-null   int64 
 1   JSON Data    4200 non-null   object
 2   Authors      4200 non-null   int64 
 3   Changes      4200 non-null   object
 4   Text Data    4200 non-null   object
dtypes: int64(2), object(3)
memory usage: 164.2+ KB


PermissionError: [Errno 13] Permission denied: 'C:\\Users\\EBRAHIMR2\\Downloads\\Data Science Masters\\Modules\\COS 801\\pan23-multi-author-analysis-dataset1\\pan23-multi-author-analysis-dataset1-train\\output.xlsx'